In [11]:
#@title File Locations
base_dir = "" #@param {type:"string"}
state_param_file = "data/us-wa/wa_state_parameters_transpose.csv" #@param {type:"string"}
county_households_file = "data/us-wa/wa_county_household_demographics.csv" #@param {type:"string"}
county_params_file = "data/us-wa/wa_county_parameters.csv" #@param {type:"string"}
import os
if base_dir:
  state_param_file = os.path.join(base_dir, state_param_file)
  county_households_file = os.path.join(base_dir, county_households_file)
  county_params_file = os.path.join(base_dir, county_params_file)

In [15]:
import collections
import sys
import example_utils as utils
import pandas as pd
import numpy as np
import csv
LOCAL_PARAMS = set(["county_fips", "lockdown_days", "app_turned_on"])

def bucket_to_age(b):
  return int(b.split("_")[1]) // 10

def read_param_file(file_name):
  params = {}
  pf = pd.read_csv(file_name,skipinitialspace=True, comment="#")
  if "transpose" in file_name:
    for row in pf.itertuples():
      params[row.parameter_name] = row.parameter_value
  else:
    for row in pf.itertuples():
      params.update(row.as_dict())
  return params

def read_param_files(file_names):
  params = {}
  for f in file_names:
    params.update(read_param_file(f))
  return params

def build_population(params_dict, houses):
  IDs = np.arange( params_dict["n_total"], dtype='int32')
  house_no = np.zeros( IDs.shape, dtype='int32' )
  ages     = np.zeros( IDs.shape , dtype='int32' )
  idx = 0
  house_idx = 0
  for house in houses.itertuples():
    house_pop = 0
    for col in houses.columns:
      if col.startswith("a_"):
        age = bucket_to_age(col)
        cnt = int(getattr(house,col))
        if cnt == 0:
          continue
        house_pop += cnt
        house_no[idx:idx+cnt] = house_idx
        ages[idx:idx+cnt] = age
        idx += cnt
    if house_pop != 0:
      house_idx += 1
  return pd.DataFrame({'ID':IDs, 'age_group':ages, 'house_no':house_no})

def run_model(params_dict, houses):
  total_days_left = int(params_dict['end_time'])

  params  = utils.get_baseline_parameters()
  for p, v in params_dict.items():
    if p in LOCAL_PARAMS:
      continue
    if isinstance(v, np.int64) or (hasattr(v, "is_integer") and v.is_integer()):
      params.set_param( p, int(v) )
    else:
      params.set_param( p, v )
  params.set_demographic_household_table( build_population(params_dict, houses) )
  model = utils.get_simulation( params ).env.model

  m_out = []
  model.one_time_step()
  m_out.append(model.one_time_step_results())
  total_days_left -= 1
  while m_out[-1]["total_infected"] < params_dict["n_total"] * 0.01:
      model.one_time_step()
      m_out.append(model.one_time_step_results())
      total_days_left -= 1

  params_dict["time_offset"] = params_dict['end_time'] - total_days_left

  model.update_running_params("lockdown_on", 1)
          
  for step in range(total_days_left):
      if step == params_dict['lockdown_days']:
          model.update_running_params("lockdown_on", 0)
          
      model.one_time_step()
      m_out.append(model.one_time_step_results())

  df = pd.DataFrame( m_out )
  m_out = []
  del model
  return params_dict, df

def run_counties(county_params, all_households, params_files=[], params_overrides={}, counties=None):
  if counties is None:
    counties = county_params.county_fips.unique()
  base_params = params_overrides
  base_params.update(read_param_files(params_files))
  outputs = []
  for county in counties[:1]:
    params = base_params.copy()
    params.update(county_params[county_params["county_fips"] == county].iloc[0])
    households = all_households[all_households["county_fips"] == county]
    outputs.append(run_model(params, households))
  return outputs

In [13]:
all_households = pd.read_csv(county_households_file)
county_params = pd.read_csv(county_params_file)
counties = county_params.county_fips.unique()l

In [17]:
local_params = {
    "lockdown_days": 35,
    "app_turned_on": 1,
}

In [ ]:
results = run_counties(county_params, all_households, params_files=[state_param_file], params_overrides=local_params, counties=counties[:1])